# Mark: 86/100


# Feedback


## Task 1: 27/30
Your solver appears to be working correctly and the animation looks right, although it doesn't run for the full 24 hours and needs a title / axes labels.

## Task 2: 14/15
Your result was accurate to within 1/10 second, which is good. Good use of the `events` parameter in `solve_ivp`.

Your result depends on the trajectory starting from a y-position of 0. To get around this, you could have found two times when the trajectory crossed the y axis (upward) and taken the difference between them.

## Task 3: 25/30
Your code produced a result that was 3s away from the correct answer. However, I was able to get this down to under 1 second by simply increasing `rtol` by an order of magnitude.

## Task 4: 10/10
Although you didn't describe it in detail, I was able to clearly see your method from reading the code, and the final plot was excellent. To see that the $x$ derivative doesn't converge to zero you could have used a log plot (of its magnitude) or plotted on two separate axes.

## Task 5: 9/10
Your result was within 0.03% of the correct one - well done! Neat and compact code. My only real complaint is the total absence of comments for this task, although your functions are named in a useful manner so that helps.

Also note that there is no point in including `r1` in your error since it is always fixed.

## Task 6: 11/15
Your code produced a result that was 1500m away from the correct one, whereas we were looking for an answer with 1000m. By playing with precision parameters I got this down to 1100m in a reasonable run-time. Still, a very good effort!

Ideally you would take the plotting part out of your `task6` function - it performs a distinct task. You should also use this plot to justify the amount of time over which you search for an impact.

## General feedback

It's great that you found the `dense_output` parameter in `solve_ivp`, although the task does not require computing a continuous solution and the `OdeSolution` object returned by `solve_ivp` was never used. There is no problem (other than perhaps speed) with writing a function that does more than the minimum required to complete the task. However, it is very important that you document this sort of thing. You should let the user know that a continuous solution is computed by `solve_ivp` in the `task1` docstring, and better yet let the user decide how to set `dense_output`.

Your commenting is mostly good. However, I'd stick to one style (preferably the one you used in `task1`, `solution_with_event`...) when it comes to docstrings.

# Checkpoint 1

**Due: Friday, 16 October, 2020 at 5:00pm BST**

### Read This First
1. Use the constants provided in the cell below. Do not use your own constants.

2. Put the code that produces the output for a given task in the cell indicated. You are welcome to add as many cells as you like for imports, function definitions, variables, etc. **Additional cells need to be in the proper order such that your code runs correctly the first time through.**

3. **IMPORTANT!** Before submitting your notebook, clear the output by clicking *Restart & Clear Output* from the *Kernel* menu. If you do not do this, the file size of your notebook will be very large.

## Libraries and Constants
Custom imports and constants should be added to a new cell.

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
from matplotlib import animation, rc
%matplotlib inline
import numpy as np
from scipy import integrate, optimize
import time

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 16

# Gravitational constant
gg     = 6.67408e-11 # m^3 s^-1 kg^-2
# Lunar mass
mass   = 7.342e22 # kg
# Lunar radius
radius = 1738000 # m
# 1 day in seconds
day    = 3600*24 # seconds

### Initial positions and velocities at t=0
rs = [1842280, 0] # m
vs = [0, 1634]    # m/s

## Equations of motion for the unperturbed case:

$
\Large
\begin{align}
\frac{d^{2} x}{dt^2} = - \frac{G M x}{(x^2 + y^2)^{3/2}}
\end{align}
$

$
\Large
\begin{align}
\frac{d^{2} y}{dt^2} = - \frac{G M y}{(x^2 + y^2)^{3/2}}
\end{align}
$

# Task 1 (30p)

In the cell below, write a function that computes the trajectory from t = 0 to tmax, where tmax is given as an argument to the function. The function should return two arrays for the x and y positions of the trajectory. Each array should have N points (equally spaced in time), where N is given as an argument to the function. You may create additional cells for defining functions.

In [ ]:
# Differential equation function to solve
# f is a list of 4 values that correspond to x, y, vx and vy
def orbit_trajectory(t, f):
    """
    f0 = x  => dx/dt  = vx
    f1 = y  => dy/dt  = vy
    f2 = vx => dvx/dt = -GM*x*(x^2 + y^2)^(-3/2)
    f3 = vy => dvy/dt = -GM*x*(x^2 + y^2)^(-3/2)
    
    Returns an array of 4 values: [x, y, vx, vy]
    """
    
    vals = np.zeros_like(f)
    vals[0] = f[2]
    vals[1] = f[3]
    vals[2] = -gg * mass * f[0] * (f[0]**2 + f[1]**2)**(-3/2)
    vals[3] = -gg * mass * f[1] * (f[0]**2 + f[1]**2)**(-3/2)
    
    return vals

In [ ]:
# Change initial conditions to a single array
fi = np.array([rs[0], rs[1], vs[0], vs[1]])

In [ ]:
def task1(N, tmax):
    """
    Compute orbital trajectory.
    
    Parameters
    ----------
    N : int
        Number of points in trajectory arrays
    tmax : float
        End time of integration in units of seconds.
        
    Returns
    -------
    x : array
        x positions of the trajectory
    y : array
        y positions of the trajectory
    """
    # Set t values
    ts = np.linspace(0, tmax, N)
    trange = (ts[0], ts[-1])
    
    sol = integrate.solve_ivp(orbit_trajectory, trange, fi, t_eval=ts, 
                              dense_output=True, rtol = 1e-5)
    
    return sol.y[0], sol.y[1]

## Testing task 1

The cell below will run your function with inputs of tmax = 1 day (in seconds) and some number of points. The assert statements below will test that the returned arrays are the correct size.

In [ ]:
t_max = day
n_points = int(t_max / 100)

x_pos, y_pos = task1(n_points, t_max)

assert x_pos.size == n_points
assert y_pos.size == n_points

# Task 1 continued

In the cell below, create an animation of the spacecraft's trajectory for t = 0 to 24 hours that includes a circle representing the Moon. Each frame of the animation should only show the last few points to avoid overlapping a previous orbit. A successful animation will be worth the full 30 points. Alternatively, create a static plot showing the Moon and the spacecraft's trajectory. This will be worth a maximum of 25 points.

In [ ]:
# Setting up the figure, the axis, and the plot element
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xlim(-2.2e6, 2.2e6)
ax.set_ylim(-2.2e6, 2.2e6)
line, = ax.plot([], [], lw=2)

In [ ]:
# Add the moon
circle = plt.Circle((0, 0), radius, color='gray', alpha=0.8)
ax.add_artist(circle)

In [ ]:
# Data used to animate the plot is already calculated
# Positions stored in x_pos and y_pos
def init():
    line.set_data(x_pos[0:5], y_pos[0:5])
    return (line,)

# Define an animation function
def animate(i):
    x = x_pos[i:i+3]
    y = y_pos[i:i+3]
    line.set_data(x, y)
    return (line,)

In [ ]:
anim = animation.FuncAnimation(fig, animate, frames=n_points//3, interval=20, blit=True)

HTML(anim.to_jshtml())

# Task 2 (15p)

Determine the orbital period T. Your solution must be calculated numerically, i.e., not simply using the analytical expression. The obtained value must be within +/-1 s of the correct value.

In the cell below, write a function that returns the orbital period in units of seconds.

In [ ]:
def complete_orbit(t, f):
    return f[1]
complete_orbit.direction = 1

In [ ]:
def solution_with_event(N, tmax):
    """
    Compute orbital trajectory.
    
    Parameters
    ----------
    N : int
        Number of points in trajectory arrays
    tmax : float
        End time of integration in units of seconds.
        
    Returns
    -------
    Solution object of the scipy.integrate.solve_ivp class
    Defined atributes:
        t - returns the time points
        y - values of the solution at t
        t_events - times at which the seeked event happens
    """
    # Set t values
    ts = np.linspace(0, tmax, N)
    trange = (ts[0], ts[-1])
    
    sol = integrate.solve_ivp(orbit_trajectory, trange, fi, events=(complete_orbit),
                              t_eval=ts, dense_output=True, rtol = 1e-6)
    
    
    return sol

In [ ]:
t_max = day
n_points = int(t_max/100)

In [ ]:
def task2():
    sol = solution_with_event(n_points, t_max)
    # Take the second event because the first one is the first iteration, where y = 0
    return sol.t_events[0][1]

print(task2())

## Testing task 2

The cell below will run your function and compare with the analytic answer. They should agree to within 1 second.

In [ ]:
T_calc = task2()
t1 = time.time()
print (f"Calculated orbital period: {T_calc} seconds.")
t2 = time.time()

mu = gg * mass
T_analytic = 2 * np.pi * np.sqrt((rs[0]**3 * mu**2) / (2*mu - rs[0]*vs[1]**2)**3)

print (f"Difference with correct answer: {T_calc - T_analytic} seconds.")
print (f"Solution calculated in {t2-t1} seconds.")

assert abs(T_calc - T_analytic) <= 1

print ("Hooray!")

# Task 3 (20p)

Now add a correction that makes the gravitational field non-spherical. The correction rotates with the Moon (one full rotation every T$_{Moon}$ = 27.3 days). How long does it take until the spacecraft hits the Moon? The time must be accurate to +/-1 s. Assume the Moon’s surface is a sphere. The equations of motion now become:

$
\Large
\begin{align}
\frac{d^{2} x}{dt^2} = - \frac{G M x}{(x^2 + y^2)^{3/2}}
- \frac{q\ G M x^\prime}{(x^{^\prime2} + y^{^\prime2})^{3/2}}
\end{align}
$

$
\Large
\begin{align}
\frac{d^{2} y}{dt^2} = - \frac{G M y}{(x^2 + y^2)^{3/2}}
- \frac{q\ G M y^\prime}{(x^{^\prime2} + y^{^\prime2})^{3/2}}
\end{align}
$

$
\Large
\begin{align}
x^\prime = x + 0.8\ R \cos \left( \frac{2 \pi t}{T_{Moon}} \right)
\end{align}
$

$
\Large
\begin{align}
y^\prime = y + 0.8\ R \sin \left( \frac{2 \pi t}{T_{Moon}} \right)
\end{align}
$

where q = 0.00025.

## Task 3 objectives:
1. Compute the time at which the spacecraft hits the Moon.
2. Make a plot of the height of the spacecraft above the Moon's surface as a function of time. Don't forget to label axes and include units.

# Task 3 part 1

In the cell below, create a function that returns the impact time accurate to within 1 second.

In [ ]:
q = 0.00025
Tmoon = 27.3 * day

In [ ]:
# Change initial conditions to a single array
fi = np.array([rs[0], rs[1], vs[0], vs[1]])

In [ ]:
def nonspherical_trajectory(t, f):
    """
    x' = x + 0.8 * radius * cos(2*pi*t/Tmoon)
    y' = y + 0.8 * radius * sin(2*pi*t/Tmoon)
    
    f0 = x  => dx/dt  = vx
    f1 = y  => dy/dt  = vy
    f2 = vx => dvx/dt = -GMx*(x^2+y^2)^(-3/2) - qGM*x'*(x'^2+y'^2)^(-3/2)
    f3 = vy => dvy/dt = -GMy*(x^2+y^2)^(-3/2) - qGM*y'*(y'^2+y'^2)^(-3/2)
    
    Returns an array of 4 values: [x, y, vx, vy]
    """
    
    x1 = f[0] + 0.8 * radius * np.cos(2*np.pi*t/Tmoon)
    y1 = f[1] + 0.8 * radius * np.sin(2*np.pi*t/Tmoon)
    
    vals = np.zeros_like(f)
    vals[0] = f[2]
    vals[1] = f[3]
    vals[2] = -gg*mass*f[0]*(f[0]**2+f[1]**2)**(-3/2) - q*gg*mass*x1*(x1**2+y1**2)**(-3/2)
    vals[3] = -gg*mass*f[1]*(f[0]**2+f[1]**2)**(-3/2) - q*gg*mass*y1*(x1**2+y1**2)**(-3/2)
    
    return vals

In [ ]:
def satellite_crashes(t, f):
    return np.sqrt(f[0]**2 + f[1]**2) - radius
satellite_crashes.terminal = True

In [ ]:
def solution_nonspherical(N, tmax):
    """
    Compute orbital trajectory.
    
    Parameters
    ----------
    N : int
        Number of points in trajectory arrays
    tmax : float
        End time of integration in units of seconds.
        
    Returns
    -------
    Solution object of the scipy.integrate.solve_ivp class
    Defined atributes:
        t - returns the time points
        y - values of the solution at t
        t_events - times at which the seeked event happens
    """
    # Set t values
    ts = np.linspace(0, tmax, N)
    trange = (ts[0], ts[-1])
    
    sol = integrate.solve_ivp(nonspherical_trajectory, trange, fi, 
                              events=(satellite_crashes), t_eval=ts, 
                              dense_output=True, rtol = 1e-7)
    
    return sol

In [ ]:
t_max = Tmoon
n_points = int(t_max / 100)

In [ ]:
def task3():
    sol = solution_nonspherical(n_points, t_max)
    return sol.t_events[0][0]

## Testing task 3 part 1

The cell below will run your function and print your answer. This will be tested against the correct answer (not given).

In [ ]:
t1 = time.time()
t_impact = task3()
t2 = time.time()

print (f"Time to impact: {t_impact:.2f} seconds ({t_impact / day:.2f} days).")
print (f"Solution calculated in {t2-t1} seconds.")
### BEGIN HIDDEN TESTS
t_correct = 627064.2858659648

print (f"Difference with correct answer: {t_impact - t_correct} seconds.")

assert abs(t_impact - t_correct) <= 1
### END HIDDEN TESTS

## Task 3 part 2

In the cell below, plot of the height of the spacecraft above the Moon's surface as a function of time. Don't forget to label axes and include units.

In [ ]:
sol = solution_nonspherical(n_points, t_max)
heights = np.sqrt(sol.y[0]**2 + sol.y[1]**2) - np.ones_like(sol.y[0]) * radius

In [ ]:
plt.plot(sol.t, heights)
plt.title('Height of the spacecraft as a function of time')
plt.xlabel('t [s]')
plt.ylabel('Height above the surface [m]')

# Task 4 (10p)

Which coordinate (x or y) of the position of the spacecraft after one revolution (orbital period T from task 2) is more sensitive to small changes in the amplitude of the correction? To answer this, calculate the derivatives of dx/dq and dy/dq at t = T, for q = 0. Write your answer in the cell below, describing how you arrived at it. Place any code that demonstrates your solution in the cell with the function called `task4`.

YOUR ANSWER HERE

The derivative is defined as $\lim_{h\to 0} \frac{f(x + h) - f(x)}{h}$. \
This can be approximated with a sufficiently small h as $\frac{f(x + \delta h) - f(x)}{\delta h}$.

$\frac{dx}{dq}$ and $\frac{dy}{dq}$ then becomes $\frac{x(q + \delta q) - x(q)}{\delta q}$ and $\frac{y(q + \delta q) - y(q)}{\delta q}$.

In [ ]:
delta_q = 1e-5
q = 0
mu = gg * mass
T_analytic = 2 * np.pi * np.sqrt((rs[0]**3 * mu**2) / (2*mu - rs[0]*vs[1]**2)**3)

In [ ]:
def nonspherical_trajectory_based_on_q(t, f, q):
    x1 = f[0] + 0.8 * radius * np.cos(2*np.pi*t/Tmoon)
    y1 = f[1] + 0.8 * radius * np.sin(2*np.pi*t/Tmoon)
    
    vals = np.zeros_like(f)
    vals[0] = f[2]
    vals[1] = f[3]
    vals[2] = -gg*mass*f[0]*(f[0]**2+f[1]**2)**(-3/2) - q*gg*mass*x1*(x1**2+y1**2)**(-3/2)
    vals[3] = -gg*mass*f[1]*(f[0]**2+f[1]**2)**(-3/2) - q*gg*mass*y1*(x1**2+y1**2)**(-3/2)
    
    return vals

In [ ]:
def solution_nonspherical_based_on_q(n_points, t_max, q_var):
    
    def modified_func(t, f):
        return nonspherical_trajectory_based_on_q(t, f, q_var)
    
    ts = np.linspace(0, t_max, n_points)
    trange = (ts[0], ts[-1])
    
    sol = integrate.solve_ivp(modified_func, trange, fi, 
                              t_eval=ts, dense_output=True, rtol = 1e-7)
        
    return sol

Calculate the solution for $q=0$ first.

In [ ]:
t_max = T_analytic
n_points = int(t_max / 100)

In [ ]:
sol1 = solution_nonspherical_based_on_q(n_points, t_max, q)

Now calculate the solution for $q = 0 + \delta q$.

In [ ]:
q = q + delta_q

In [ ]:
sol2 = solution_nonspherical_based_on_q(n_points, t_max, q)

Finally, find the approximated derivative at a time T.

In [ ]:
def task4():
    delta_x = sol2.y[0][-1] - sol1.y[0][-1]
    delta_y = sol2.y[1][-1] - sol1.y[1][-1]
    
    return delta_x / delta_q, delta_y / delta_q

print(task4())

Y coordinate seems to be much more sensitive to a small change in $q$.

In [ ]:
def get_derivatives(q0, delta_qs):
    dx_dqs = np.zeros_like(delta_qs)
    dy_dqs = np.zeros_like(delta_qs)
    
    for i in range(len(delta_qs)):
        sol1 = solution_nonspherical_based_on_q(n_points, t_max, q0)
        sol2 = solution_nonspherical_based_on_q(n_points, t_max, q0 + delta_qs[i])
        
        delta_x = sol2.y[0][-1] - sol1.y[0][-1]
        delta_y = sol2.y[1][-1] - sol1.y[1][-1]
        
        dx_dqs[i] = delta_x / delta_qs[i]
        dy_dqs[i] = delta_y / delta_qs[i]
        
    return dx_dqs, dy_dqs

In [ ]:
q0 = 0
delta_qs = np.logspace(-10, -1, 20)
dx_dqs, dy_dqs = get_derivatives(q0, delta_qs)

# Plot on a logarithmic axis
plt.semilogx(delta_qs, dx_dqs, label='dx/dq')
plt.semilogx(delta_qs, dy_dqs, label='dy/dq')
plt.legend()
plt.grid()
plt.xlabel('$\delta q$')
plt.ylabel('Derivative at $t = T$')
plt.show()


$\frac{dx}{dq}$ seems to converge to 0, this is not true. The value is just on the order of $10^5$ so it is not distinguishable at this plotting scale. Both values seem to converge, which means my calculations should be correct.

# Task 5 (10p)

The positions of the spacecraft at t=0, t=T/2, and t=T are given in the cell below. Use them to determine the amplitude of the correction q. Note, this is a different value than for the previous tasks.

A comment for those interested in space science: this is a highly simplified and unrealistic version of the task NASA scientists had to carry out to map out the gravity at the Moon's surface using "telemetry" data (positions and velocities) of various spacecrafts orbiting the Moon.

Put your code in the cell that starts with `def task5():`.

Your answer should be within 20% of the correct answer.

In [ ]:
r1 = [1842280.0, 0.0]
r2 = [-1856332.7223839264, -717.5195460640389]
r3 = [1842271.070055315, 3847.378923359429]

In [ ]:
r1 = np.array(r1)
r2 = np.array(r2)
r3 = np.array(r3)

In [ ]:
mu = gg * mass
T_analytic = 2 * np.pi * np.sqrt((rs[0]**3 * mu**2) / (2*mu - rs[0]*vs[1]**2)**3)

In [ ]:
t_max = T_analytic
n_points = int(t_max / 100)

In [ ]:
def function_to_optimize(q):
    sol = solution_nonspherical_based_on_q(n_points, t_max, q)
    
    pos1 = np.array([sol.y[0][0], sol.y[1][0]])
    pos2 = np.array([sol.y[0][n_points//2], sol.y[1][n_points//2]])
    pos3 = np.array([sol.y[0][-1], sol.y[1][-1]])
    
    error1 = abs(np.linalg.norm(pos1 - r1)) / np.linalg.norm(r1)
    error2 = abs(np.linalg.norm(pos2 - r2)) / np.linalg.norm(r2)
    error3 = abs(np.linalg.norm(pos3 - r3)) / np.linalg.norm(r3)
    
    return error1 + error2 + error3

In [ ]:
def task5():
    return optimize.minimize_scalar(function_to_optimize).x

Want to create a function that takes in a certain q and returns the total relative error.

## Testing task 5

The cell below will run your function and print your answer. This will be tested against the correct answer (not given). Your answer should be within 20% of the correct answer.

In [ ]:
t1 = time.time()
mystery_q = task5()
t2 = time.time()

print (f"q = {mystery_q}")
print (f"Solution calculated in {t2-t1} seconds.")

### BEGIN HIDDEN TESTS
q_correct = 0.00012

print (f"Difference with correct answer: {100*abs(mystery_q/q_correct-1)}%.")

assert abs(mystery_q/q_correct-1) <= 0.2
### END HIDDEN TESTS

I accidentally copied and pasted the cell above, and I can't delete it.

In [ ]:
t1 = time.time()
mystery_q = task5()
t2 = time.time()

print (f"q = {mystery_q}")
print (f"Solution calculated in {t2-t1} seconds.")

### BEGIN HIDDEN TESTS
q_correct = 0.00012

print (f"Difference with correct answer: {100*abs(mystery_q/q_correct-1)}%.")

assert abs(mystery_q/q_correct-1) <= 0.2
### END HIDDEN TESTS

# Task 6 (15p)

What is the minimum initial height of a circular orbit such that, for the perturbation from task 3 (q = 0.00025), the spacecraft does not collide with the Moon but remains gravitationally bound to it?

The orbit may still show oscillations as in task 3, but the spacecraft cannot not hit the lunar surface.

Create a function `task6` that returns the minimum height of the circular orbit in units of meters. Explain your approach. To obtain full marks, the answer must be correct to +/-1 km.

The following formula for the velocity of a point mass in circular orbit of radius r, orbiting a spherically symmetric body of mass M, may be useful:

$
\Large
\begin{align}
v_{c} = \sqrt{\frac{G\ M}{r}}.
\end{align}
$

In the cell below, create a function that calculates the minimum height of a stable orbit in units of meters. Your answer should be within 1000 meters of the correct answer.

My approach to this problem is calculating a bunch of orbits and using the optimize.optimize_scalar function to find the lowest orbit. My assumption is that if the satellite doesn't crash in month time (time the nonspherical gravity changes makes one circle) it wont crash at all.

In [ ]:
q = 0.00025
# Change initial conditions to a single array
fi = np.array([rs[0], rs[1], vs[0], vs[1]])

In [ ]:
def current_EOM(t, f):
    return nonspherical_trajectory_based_on_q(t, f, q)

In [ ]:
t_max = 1.1 * Tmoon
n_points = int(t_max / 100000)
ts = np.linspace(0, t_max, n_points)
trange = (ts[0], ts[-1])

In [ ]:
def function_to_find0(initial_radius):
    initial_velocity = np.sqrt(gg*mass / initial_radius)
    f = np.array([initial_radius, 0, 0, initial_velocity])
    
    sol = integrate.solve_ivp(current_EOM, trange, f,
                              events=(satellite_crashes), t_eval=ts,
                                dense_output=True, rtol=1e-6)
    
    # Spacecraft that crash before Tmoon return a positive value, and those that do so after, or not at all return
    # a negative value. Thats how the brentq method converges to a final value
    return Tmoon - sol.t[-1]

In [ ]:
def task6():
    r0 = optimize.brentq(function_to_find0, rs[0], 1.2 * rs[0], rtol=1e-3)
    
    # Make a plot of the spacecraft
    
    # Redefine time values so the plot looks nicer
    n_points = int(t_max / 1000)
    ts = np.linspace(0, t_max, n_points)
    trange = (ts[0], ts[-1])
    
    initial_radius = r0
    initial_velocity = np.sqrt(gg*mass / initial_radius)
    f = np.array([initial_radius, 0, 0, initial_velocity])
    
    sol = integrate.solve_ivp(current_EOM, trange, f,
                            events=(satellite_crashes), t_eval=ts,
                            dense_output=True, rtol=1e-6)
    heights = np.sqrt(sol.y[0]**2 + sol.y[1]**2) - np.ones_like(sol.y[0]) * radius
    plt.plot(sol.t, heights)
    plt.xlabel('Time [s]')
    plt.ylabel('Height[m]')
    plt.title('Height of the Spacecraft vs Time')
    
    return r0 - radius

## Testing task 6

The cell below will run your function and print your answer. This will be tested against the correct answer (not given). Your answer should be within 1000 meters of the correct answer.

In [ ]:
t1 = time.time()
min_height = task6()
t2 = time.time()
print (f"Minimum height of stable orbit: {min_height} m.")
print (f"Solution calculated in {t2-t1} seconds.")

### BEGIN HIDDEN TESTS
min_height_correct = 154047.6006164552

print (f"Difference with correct answer: {min_height - min_height_correct} meters.")

assert abs(min_height - min_height_correct) <= 1000
### END HIDDEN TESTS